In [1]:
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

# model_path = "word2vec_model.model"
# model = Word2Vec.load(model_path)

model_path = 'C:\\Users\\basil\\Downloads\\wordvecs\\model.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(
    model_path, binary=True)

In [14]:
import spacy
import math
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [121]:
from gensim.matutils import hellinger

def normalise(inputWord):
    if inputWord in model.key_to_index:
        x = nlp(inputWord)[0].lemma_
        return (x if x in model.key_to_index else inputWord)
    return (inputWord)


def score(targetWord, inputWord):

    alpha = 0.5
    beta = 0.5
    initial_score = 1000
    wordDistance =  model.wmdistance(inputWord, targetWord)
    wordSimilarity = (1 + model.similarity(inputWord, targetWord))/2
    weighted_score = (1 - ((alpha * (1 - wordDistance) + beta * wordSimilarity)))*initial_score

    # a = (2 - (wordSimilarity/wordDistance))

    # if a == 1:
    #     return(1)
    # elif a>1 and a<2:
    #     return(int(5*a))
    # elif a>2 and a<3:
    #     return(int(50*a))
    # else:
    #     return (int(abs(a*100)))
    return( wordDistance, wordSimilarity, int(weighted_score)+1)

def fetchBestFitHint(lowestScore, targetWord, closestWord):
    word = None
    positive = []
    positive.append(targetWord)
    if closestWord != None and closestWord != '':
        positive.append(closestWord)
    hintWords = [i for i in dict(
        model.most_similar(positive=positive, topn=30))]
    low = 1
    for i in hintWords:
        if str(i).isalpha() and str(i) != targetWord and str(i) not in targetWord and targetWord not in str(i):
            scoreCurrent = int(score(targetWord, normalise(i)))
            print(normalise(i), scoreCurrent, lowestScore)
            if scoreCurrent < lowestScore:
                if low < scoreCurrent:
                    word = normalise(i)
                    low = scoreCurrent
    return (word, low)

In [24]:
from flask import Flask, render_template, request, jsonify


app = Flask(__name__)

targetWord = 'gamble'
entries = {}
inputWord = None
closestWord = ''
latestWord = None
lowestScore = None
message = None


@app.route('/reset', methods=['GET', 'POST'])
def reset():

    global closestWord
    global entries
    global lowestScore
    global message
    global inputWord
    global latestWord
    global targetWord

    targetWord = 'gamble'
    entries = {}
    inputWord = None
    closestWord = ''
    latestWord = None
    lowestScore = None
    message = None
    return render_template('index.html', entry=entries, current=latestWord if latestWord != None else None, currentValue=entries[latestWord] if latestWord != None else None, message=message)


@app.route('/hint', methods=['GET', 'POST'])
def getHint():

    global closestWord
    global entries
    global lowestScore
    global latestWord
    message = None
    hintWord, score = fetchBestFitHint(
        int(lowestScore) if lowestScore != None else 999999, targetWord, closestWord)

    if hintWord == None:
        message = "You are closer that you think!"
    else:
        entries[hintWord] = score
        lowestScore = score
        latestWord = hintWord
        closestWord = hintWord
        entries = dict(sorted(entries.items(), key=lambda item: item[1]))
    return render_template('index.html', entry=entries, current=hintWord, currentValue=entries[hintWord] if hintWord != None else None, message=message)


@app.route('/giveUp', methods=['GET', 'POST'])
def giveUp():
    global message
    global entries

    entries[targetWord] = 0
    entries = dict(sorted(entries.items(), key=lambda item: item[1]))
    return render_template('index.html', entry=entries, current=None, currentValue=None, message=None)


@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':

        global inputWord
        global latestWord
        global lowestScore
        global entries
        global message

        inputWord = normalise(request.form.get('inputWord'))
        try:
            message = None
            scoreValue = int(score(targetWord, inputWord))
            entries[inputWord] = scoreValue
            entries = dict(sorted(entries.items(), key=lambda item: item[1]))
            latestWord = inputWord
            if (lowestScore == None or lowestScore >= scoreValue) and inputWord in model.key_to_index:
                lowestScore = scoreValue
                global closestWord
                closestWord = inputWord
        except Exception as e:
            print(e)
            message = "Entry not in list"
            entries = dict(sorted(entries.items(), key=lambda item: item[1]))
    return render_template('index.html', entry=entries, current=latestWord if latestWord != None else None, currentValue=entries[latestWord] if latestWord != None else None, message=message)


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [15/Oct/2023 11:33:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2023 11:33:10] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2023 11:33:15] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2023 11:33:21] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2023 11:33:26] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2023 11:33:28] "POST /reset HTTP/1.1" 200 -
